# How to use different estimators and how to interpret the results

In [ ]:
import pandas  as pd
import biogeme.database  as db
import biogeme.biogeme  as bio
import biogeme.optimization as opt
from IPython.display import display, HTML

**Import Swissmetro data**

In [ ]:
pandas = pd.read_csv("data/swissmetro.dat",sep='\t')
database = db.Database("data/swissmetro", pandas)

**Use collumn names as variables**

In [ ]:
from headers import *

**Exclude some unwanted entries**

In [ ]:
exclude = (( PURPOSE != 1 ) * ( PURPOSE != 3 ) + ( CHOICE == 0 )) > 0

database.remove(exclude)

**Define some dummy variables**

In [ ]:
SM_COST = SM_CO * ( GA == 0 )
TRAIN_COST = TRAIN_CO * ( GA == 0 )

CAR_AV_SP = DefineVariable ('CAR_AV_SP', CAR_AV * ( SP !=0 ), database)
TRAIN_AV_SP = DefineVariable ('TRAIN_AV_SP', TRAIN_AV * ( SP != 0 ), database)

**Rescale some data**

In [ ]:
TRAIN_TT_SCALED   = DefineVariable('TRAIN_TT_SCALED',   TRAIN_TT / 100.0, database)
TRAIN_COST_SCALED = DefineVariable('TRAIN_COST_SCALED', TRAIN_COST / 100, database)
SM_TT_SCALED      = DefineVariable('SM_TT_SCALED',      SM_TT / 100.0   , database)
SM_COST_SCALED    = DefineVariable('SM_COST_SCALED',    SM_COST / 100   , database)
CAR_TT_SCALED     = DefineVariable('CAR_TT_SCALED',     CAR_TT / 100    , database)
CAR_CO_SCALED     = DefineVariable('CAR_CO_SCALED',     CAR_CO / 100    , database)

**Create parameters to be estimated**

In [ ]:
ASC_CAR = Beta('ASC_CAR',0,None ,None ,0)
ASC_TRAIN = Beta('ASC_TRAIN',0,None ,None ,0)
ASC_SM = Beta('ASC_SM',0,None ,None ,1)
B_TIME = Beta('B_TIME',0,None ,None ,0)
B_COST = Beta('B_COST',0,None ,None ,0)

**Define the utility functions**

In [ ]:
V1 = ASC_TRAIN + \
     B_TIME * TRAIN_TT_SCALED + \
     B_COST * TRAIN_COST_SCALED
V2 = ASC_SM + \
     B_TIME * SM_TT_SCALED + \
     B_COST * SM_COST_SCALED
V3 = ASC_CAR + \
     B_TIME * CAR_TT_SCALED + \
     B_COST * CAR_CO_SCALED

**Associate utility functions with alternatives and associate availability of alternatives**

In [ ]:
V = {1: V1,
     2: V2,
     3: V3}

av = {1: TRAIN_AV_SP,
      2: SM_AV,
      3: CAR_AV_SP}

**Define the model**

In [ ]:
logprob = bioLogLogit(V,av,CHOICE)

**Define the Biogeme object**

In [ ]:
biogeme  = bio.BIOGEME(database, logprob)

biogeme.modelName = "swissmetro_logit_estimators"

**Define the algorithms to estimat the maximum likelihood**

In [ ]:
algos={'CFSQP':None,
       'scipy':opt.scipy,
       'Line search':opt.newtonLineSearchForBiogeme,
       'Trust region (dogleg)':opt.newtonTrustRegionForBiogeme,
       'Trust region (cg)':opt.newtonTrustRegionForBiogeme}

algoParameters ={'Trust region (dogleg)':{'dogleg':True},
                 'Trust region (cg)':{'dogleg':False}}

**Estimate the model**

In [ ]:
biogeme.generateHtml = False
biogeme.generatePickle = False

results_algo = {}
for name,algo in algos.items():
    p = algoParameters.get(name)
    results_algo[name] = biogeme.estimate(algorithm=algo,algoParameters=p)
    g = results_algo[name].data.g
    biogeme.createLogFile()

**Print results for the different algorithms**

In [ ]:
print("Algorithm             loglike         normg    time          feval diagnostic")
print("+++++++++             +++++++         +++++    ++++          +++++ ++++++++++")
for name,algo in algos.items():
    print(f"{name:21} {results_algo[name].data.logLike:10.7f} {np.inner(g,g):10.3g} {results_algo[name].data.optimizationTime} {results_algo[name].data.numberOfFunctionEval:4} {results_algo[name].data.optimizationMessage}")



**Print results**

In [ ]:
results_1 = results_algo['scipy']
results_2 = results_algo['CFSQP']
results_3 = results_algo['Line search']

betas_1 = results_1.getBetaValues()
betas_2 = results_2.getBetaValues()
betas_3 = results_3.getBetaValues()

for k in betas_1.keys():
    b_1 = betas_1[k]
    b_2 = betas_2[k]
    b_3 = betas_3[k]
    print(f"{k:10}=\t{b_1:.8g}\t{b_2:.8g}\t{b_3:.8g}")

**Explore the results**

In [ ]:
results = results_algo['scipy']

Results = results.getEstimatedParameters()
display(Results)

**Get the general statistics**
* Number of estimated parameters($K$)
* Sample size ($N$)
* Number of excluded observations
* Log likelihood of the sample with the default values for the parameters ($\mathcal{L}^i)$)
* Log likelihood for the final estimated model ($\mathcal{L}^*)$)
* Likelihood ratio:
\begin{align}
-2 (\mathcal{L}^i-\mathcal{L}^*)
\end{align}
* Rho-square for the init model
\begin{align}
\rho^2 = 1- \frac{\mathcal{L}^*}{\mathcal{L}^i}
\end{align}
* Rho-square adjusted for the init model
\begin{align}
\rho^2 = 1- \frac{\mathcal{L}^* - K}{\mathcal{L}^i}
\end{align}
* Akaike Information Criterion
\begin{align}
2 K - 2 \mathcal{L}^*
\end{align}
* Bayesian Information Criterion
\begin{align}
K N- 2 \mathcal{L}^*
\end{align}
* Final gradient norm
* Iterations
* Optimization Time

In [ ]:
gs = results.getGeneralStatistics()

for k,v in gs.items():
    print("{}= {}".format(k.ljust(45),v[0]))
# print(gs)

**Clean up output files**

In [ ]:
import glob, os

result_files = glob.glob(biogeme.modelName+'*')
result_files = [x for x in result_files if x != biogeme.modelName+'.ipynb']
if len(result_files) != 0:
    result_dir = "results"
    print('Moving the following files:')
    for result_file in result_files:
        print('\t',result_file)
        os.rename(result_file, os.path.join(result_dir, result_file))